In [53]:
import torch.nn as nn
import torch.nn.functional as F


class ppNet(nn.Module):
    def __init__(self):
        super(ppNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 26, 5)
        self.conv4 = nn.Conv2d(26, 36, 5)
        self.bn3 = nn.BatchNorm2d(26)
        self.bn4 = nn.BatchNorm2d(36)
        self.fc1 = nn.Linear(36*10*10, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 36*10*10)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = ppNet()

In [54]:
net

ppNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 26, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(26, 36, kernel_size=(5, 5), stride=(1, 1))
  (bn3): BatchNorm2d(26, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3600, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)

In [55]:

from __future__ import print_function, division
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import time
import random
%matplotlib inline


import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms


train_Aug = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),       
                                            torchvision.transforms.ToTensor()])
test_Aug = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),  
                                           torchvision.transforms.ToTensor()])

In [56]:
train_dir = "C:\\Users\\91958\\elocations_dataset\\train"


trainset = torchvision.datasets.ImageFolder(train_dir,transform = train_Aug)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=40, shuffle=True, num_workers=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [57]:
class_names = trainset.classes
len(class_names)

2

In [58]:

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(net.to(device).parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [59]:
model_dir = 'C:\\Users\\91958\\custom_model'
num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0): 
        exp_lr_scheduler.step()
        
        for inputs, labels in trainloader:
                inputs = inputs.to(device) 
                labels = labels.to(device)
                
                optimizer_ft.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer_ft.step()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # correct += (outputs == labels).float().sum()
        print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss.item(), correct/total))
        
        torch.save(net.state_dict(), os.path.join(model_dir, 'epoch-{}.pt'.format(epoch)))

print('Finished Training')

Epoch 1/5, Loss: 0.418, Accuracy: 0.687
Epoch 1/5, Loss: 0.270, Accuracy: 0.747
Epoch 1/5, Loss: 0.380, Accuracy: 0.773
Epoch 1/5, Loss: 0.387, Accuracy: 0.790
Epoch 1/5, Loss: 0.171, Accuracy: 0.805
Epoch 1/5, Loss: 0.266, Accuracy: 0.816
Epoch 1/5, Loss: 0.185, Accuracy: 0.827
Epoch 1/5, Loss: 0.273, Accuracy: 0.837
Epoch 1/5, Loss: 0.526, Accuracy: 0.843
Epoch 1/5, Loss: 0.225, Accuracy: 0.850
Epoch 1/5, Loss: 0.205, Accuracy: 0.855
Epoch 1/5, Loss: 0.204, Accuracy: 0.859
Epoch 1/5, Loss: 0.211, Accuracy: 0.863
Epoch 1/5, Loss: 0.138, Accuracy: 0.866
Epoch 1/5, Loss: 0.106, Accuracy: 0.870
Epoch 1/5, Loss: 0.116, Accuracy: 0.873
Epoch 1/5, Loss: 0.309, Accuracy: 0.876
Epoch 1/5, Loss: 0.201, Accuracy: 0.877
Epoch 1/5, Loss: 0.140, Accuracy: 0.879
Epoch 1/5, Loss: 0.166, Accuracy: 0.881
Epoch 1/5, Loss: 0.135, Accuracy: 0.883
Epoch 1/5, Loss: 0.252, Accuracy: 0.884
Epoch 1/5, Loss: 0.227, Accuracy: 0.886
Epoch 1/5, Loss: 0.190, Accuracy: 0.887
Epoch 1/5, Loss: 0.203, Accuracy: 0.888


Epoch 4/5, Loss: 0.181, Accuracy: 0.917
Epoch 4/5, Loss: 0.196, Accuracy: 0.917
Epoch 4/5, Loss: 0.242, Accuracy: 0.917
Epoch 4/5, Loss: 0.121, Accuracy: 0.917
Epoch 4/5, Loss: 0.481, Accuracy: 0.917
Epoch 4/5, Loss: 0.177, Accuracy: 0.917
Epoch 4/5, Loss: 0.145, Accuracy: 0.916
Epoch 4/5, Loss: 0.146, Accuracy: 0.916
Epoch 4/5, Loss: 0.270, Accuracy: 0.916
Epoch 4/5, Loss: 0.181, Accuracy: 0.917
Epoch 4/5, Loss: 0.277, Accuracy: 0.916
Epoch 4/5, Loss: 0.211, Accuracy: 0.917
Epoch 4/5, Loss: 0.353, Accuracy: 0.917
Epoch 4/5, Loss: 0.130, Accuracy: 0.917
Epoch 4/5, Loss: 0.253, Accuracy: 0.917
Epoch 4/5, Loss: 0.217, Accuracy: 0.917
Epoch 4/5, Loss: 0.129, Accuracy: 0.917
Epoch 4/5, Loss: 0.178, Accuracy: 0.917
Epoch 4/5, Loss: 0.110, Accuracy: 0.917
Epoch 5/5, Loss: 0.183, Accuracy: 0.921
Epoch 5/5, Loss: 0.144, Accuracy: 0.919
Epoch 5/5, Loss: 0.321, Accuracy: 0.919
Epoch 5/5, Loss: 0.271, Accuracy: 0.920
Epoch 5/5, Loss: 0.130, Accuracy: 0.919
Epoch 5/5, Loss: 0.184, Accuracy: 0.919


In [60]:
test_dir = "C:\\Users\\91958\\elocations_dataset\\test"
testset = torchvision.datasets.ImageFolder(test_dir,transform = test_Aug)

testloader = torch.utils.data.DataLoader(testset, batch_size=40, shuffle=True, num_workers=2)

In [61]:
net.load_state_dict(torch.load('C:\\Users\\91958\\custom_model\\epoch-4.pt'))

<All keys matched successfully>

In [62]:
net.eval()

ppNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 26, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(26, 36, kernel_size=(5, 5), stride=(1, 1))
  (bn3): BatchNorm2d(26, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3600, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)

In [63]:
num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    for i, data in enumerate(testloader, 0): 
        exp_lr_scheduler.step()
        net.eval() 
        
        for inputs, labels in testloader:
                inputs = inputs.to(device) 
                labels = labels.to(device)
                
                optimizer_ft.zero_grad()
                outputs = net(inputs)
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # correct += (outputs == labels).float().sum()
        print("Epoch {}/{}, Accuracy: {:.3f}".format(epoch+1,num_epochs, correct/total))

print('Finished Testing')

Epoch 1/5, Accuracy: 0.742
Epoch 1/5, Accuracy: 0.742
Epoch 1/5, Accuracy: 0.742
Epoch 1/5, Accuracy: 0.742
Epoch 1/5, Accuracy: 0.742
Epoch 1/5, Accuracy: 0.742
Epoch 1/5, Accuracy: 0.742
Epoch 1/5, Accuracy: 0.742
Epoch 1/5, Accuracy: 0.742
Epoch 1/5, Accuracy: 0.742
Epoch 2/5, Accuracy: 0.742
Epoch 2/5, Accuracy: 0.742
Epoch 2/5, Accuracy: 0.742
Epoch 2/5, Accuracy: 0.742
Epoch 2/5, Accuracy: 0.742
Epoch 2/5, Accuracy: 0.742
Epoch 2/5, Accuracy: 0.742
Epoch 2/5, Accuracy: 0.742
Epoch 2/5, Accuracy: 0.742
Epoch 2/5, Accuracy: 0.742
Epoch 3/5, Accuracy: 0.742
Epoch 3/5, Accuracy: 0.742
Epoch 3/5, Accuracy: 0.742
Epoch 3/5, Accuracy: 0.742
Epoch 3/5, Accuracy: 0.742
Epoch 3/5, Accuracy: 0.742
Epoch 3/5, Accuracy: 0.742
Epoch 3/5, Accuracy: 0.742
Epoch 3/5, Accuracy: 0.742
Epoch 3/5, Accuracy: 0.742
Epoch 4/5, Accuracy: 0.742
Epoch 4/5, Accuracy: 0.742
Epoch 4/5, Accuracy: 0.742
Epoch 4/5, Accuracy: 0.742
Epoch 4/5, Accuracy: 0.742
Epoch 4/5, Accuracy: 0.742
Epoch 4/5, Accuracy: 0.742
E

In [64]:
from PIL import Image
from torch.autograd import Variable

imsize = (224,224)
loader = transforms.Compose([transforms.Scale(imsize), transforms.ToTensor()])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)
    return image.cuda()

In [65]:
image = image_loader('C:\\Users\\91958\\food1.jpeg')
# print(image)
output = net(image)

In [66]:
output.cpu().data.numpy().argmax()

0

In [73]:
image1 = image_loader('C:\\Users\\91958\\humans.jpeg')
output1 = net(image1)

In [74]:
output1.cpu().data.numpy().argmax()

0

In [69]:
image2 = image_loader('C:\\Users\\91958\\19.jpeg')
# print(image)
output2 = net(image2)

In [70]:
output2.cpu().data.numpy().argmax()

1

In [71]:
image3 = image_loader('C:\\Users\\91958\\food3.jpeg')
# print(image)
output3 = net(image3)

In [72]:
output3.cpu().data.numpy().argmax()

0